# 🔵 Blue Object Shape Detection with OpenCV

This notebook demonstrates how to detect **blue-colored objects** in a video stream using OpenCV and classify their **approximate shapes** in real-time.

### ✅ Features:
- Real-time webcam capture
- HSV-based blue color filtering
- Noise removal using morphological operations
- Contour detection & shape approximation
- Shape labeling: **Rectangle** or **Circle**

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    
    lower_blue = np.array([90, 50, 70])
    upper_blue = np.array([130, 255, 255])
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 500:
            epsilon = 0.01 * cv2.arcLength(cnt, True)
            approximations = cv2.approxPolyDP(cnt, epsilon, True)

            if len(approximations) > 0:
                i, j = approximations[0][0]

                if 6 < len(approximations) < 10:
                    cv2.putText(frame, "Rectangle", (i, j), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 1)
                    x, y, w, h = cv2.boundingRect(cnt)
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                elif len(approximations) > 10:
                    cv2.putText(frame, "Circle", (i, j), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 1)
                    center, radius = cv2.minEnclosingCircle(cnt)
                    center = int(center[0]), int(center[1])
                    radius = int(radius)
                    cv2.circle(frame, center, radius, (0, 0, 255), 2)


    cv2.imshow('Blue Object Detection', frame)
    cv2.imshow('Mask', mask)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()